# Kaggle

In [1]:
import os
import json

with open("/Users/g-ogaki/workspace/kaggle.json") as f:
    json_data = json.load(f) 
    os.environ["KAGGLE_USERNAME"] = json_data["username"]
    os.environ["KAGGLE_KEY"] = json_data["key"]

In [2]:
!kaggle competitions download -c tabular-playground-series-dec-2021

^C
User cancelled operation


In [6]:
!unzip tabular-playground-series-dec-2021.zip

Archive:  tabular-playground-series-dec-2021.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


# Import

In [2]:
from utils import init_logger, timer, fix_seed
import random
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import numpy as np
import category_encoders as ce
import matplotlib.pyplot as plt
import datetime as dt
from tqdm import tqdm
from tqdm.notebook import tqdm_notebook
tqdm_notebook.pandas(desc="Processing:")

fix_seed()
logger = init_logger()

/opt/homebrew/lib/python3.9/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
PROJECT_NAME = "v1"

# Load

In [4]:
from etl import load_train_data, load_test_data

In [5]:
with timer("read csv", logger):
    df = load_train_data()
    df_test = load_test_data()

2021/12/31 14:33:06 45 [INFO] [read csv] start.
2021/12/31 14:33:16 47 [INFO] [read csv] done in 9.592 seconds.


In [6]:
X = df.drop(["Id", "Cover_Type"], axis=1)
y = df["Cover_Type"] - 1
X_test = df_test.drop(["Id"], axis=1)

# Training

In [7]:
from model_lgb import ModelLGB
from runner import Runner
from hyperopt import hp, fmin, tpe, space_eval

In [8]:
runner = Runner(PROJECT_NAME, ModelLGB, cv=False)

In [9]:
space = {
    "objective": "multiclass",
    "num_classes": 7,
    "metric": ["multi_logloss", "multi_error"],
    "max_leaves": 30 + hp.randint("_max_leaves", 40),
    "min_data_in_leaf": 10 + hp.randint("_min_data_in_leaf", 40),
    "lambda_l1": hp.loguniform("lambda_l1", np.log(0.01), np.log(1)),
    "lambda_l2": hp.loguniform("lambda_l2", np.log(0.01), np.log(1)),
    "bagging_fraction": hp.uniform("bagging_fraction", 0.7, 0.9),
    "feature_fraction": hp.uniform("feature_fraction", 0.7, 0.9),
    "learning_rate": 0.01,
    "seed": 0,
    "verbose": -1,
}

train_params = {
    "num_boost_round": 2000,
    "early_stopping_rounds": 20,
    "verbose_eval": 50,
}

In [10]:
def objective(args):
    logger.info(args)
    runner.train(X, y, args, train_params)
    return runner.get_score()

In [ ]:
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=20)
best_params = space_eval(space, best)
logger.info("best params: {}".format(best_params))

  0%|                                    | 0/20 [00:00<?, ?trial/s, best loss=?]

2021/12/31 14:33:48 2 [INFO] {'bagging_fraction': 0.8041882760340158, 'feature_fraction': 0.758404881916913, 'lambda_l1': 0.8732061538314787, 'lambda_l2': 0.02853921475938039, 'learning_rate': 0.01, 'max_leaves': 33, 'metric': ('multi_logloss', 'multi_error'), 'min_data_in_leaf': 41, 'num_classes': 7, 'objective': 'multiclass', 'seed': 0, 'verbose': -1}
/opt/homebrew/lib/python3.9/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(

/opt/homebrew/lib/python3.9/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "

/opt/homebrew/lib/python3.9/site-packages/lightgbm/engine.py:239: UserWarning: 'verbo

Training until validation scores don't improve for 20 rounds                    
[50]	training's multi_logloss: 0.45077	training's multi_error: 0.0803313	valid_1's multi_logloss: 0.45129	valid_1's multi_error: 0.0805225
[100]	training's multi_logloss: 0.291398	training's multi_error: 0.0571466	valid_1's multi_logloss: 0.292148	valid_1's multi_error: 0.0572413
[150]	training's multi_logloss: 0.209781	training's multi_error: 0.0523722	valid_1's multi_logloss: 0.210703	valid_1's multi_error: 0.052405
[200]	training's multi_logloss: 0.166474	training's multi_error: 0.0488456	valid_1's multi_logloss: 0.167547	valid_1's multi_error: 0.0490913
[250]	training's multi_logloss: 0.141748	training's multi_error: 0.0466009	valid_1's multi_logloss: 0.142972	valid_1's multi_error: 0.0470013
[300]	training's multi_logloss: 0.12632	training's multi_error: 0.04482	valid_1's multi_logloss: 0.127683	valid_1's multi_error: 0.0453975
[350]	training's multi_logloss: 0.115918	training's multi_error: 0.0433262

2021/12/31 14:46:45 2 [INFO] {'bagging_fraction': 0.8309292802869729, 'feature_fraction': 0.8493136822063353, 'lambda_l1': 0.012037178705749304, 'lambda_l2': 0.018650660926638262, 'learning_rate': 0.01, 'max_leaves': 46, 'metric': ('multi_logloss', 'multi_error'), 'min_data_in_leaf': 19, 'num_classes': 7, 'objective': 'multiclass', 'seed': 0, 'verbose': -1}
/opt/homebrew/lib/python3.9/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(

/opt/homebrew/lib/python3.9/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "

/opt/homebrew/lib/python3.9/site-packages/lightgbm/engine.py:239: UserWarning: 'v

Training until validation scores don't improve for 20 rounds                    
[50]	training's multi_logloss: 0.419123	training's multi_error: 0.0672559	valid_1's multi_logloss: 0.420059	valid_1's multi_error: 0.0676675
[100]	training's multi_logloss: 0.25902	training's multi_error: 0.0535016	valid_1's multi_logloss: 0.260289	valid_1's multi_error: 0.0536575
[150]	training's multi_logloss: 0.184407	training's multi_error: 0.0485353	valid_1's multi_logloss: 0.185946	valid_1's multi_error: 0.0489625
[200]	training's multi_logloss: 0.146783	training's multi_error: 0.0458656	valid_1's multi_logloss: 0.148511	valid_1's multi_error: 0.0464287
[250]	training's multi_logloss: 0.124943	training's multi_error: 0.0439166	valid_1's multi_logloss: 0.12689	valid_1's multi_error: 0.0445912
[300]	training's multi_logloss: 0.112358	training's multi_error: 0.0423316	valid_1's multi_logloss: 0.11449	valid_1's multi_error: 0.0431487
[350]	training's multi_logloss: 0.104379	training's multi_error: 0.0411

...

Training until validation scores don't improve for 20 rounds                    
[50]	training's multi_logloss: 0.454084	training's multi_error: 0.0788222	valid_1's multi_logloss: 0.454689	valid_1's multi_error: 0.0792237
[100]	training's multi_logloss: 0.294272	training's multi_error: 0.0541347	valid_1's multi_logloss: 0.295186	valid_1's multi_error: 0.0543525
[150]	training's multi_logloss: 0.208092	training's multi_error: 0.048985	valid_1's multi_logloss: 0.209248	valid_1's multi_error: 0.0494125
[200]	training's multi_logloss: 0.162704	training's multi_error: 0.0458238	valid_1's multi_logloss: 0.164076	valid_1's multi_error: 0.0464613
[250]	training's multi_logloss: 0.137146	training's multi_error: 0.0438575	valid_1's multi_logloss: 0.138687	valid_1's multi_error: 0.0445888
[300]	training's multi_logloss: 0.120831	training's multi_error: 0.0423044	valid_1's multi_logloss: 0.122563	valid_1's multi_error: 0.0431038
[350]	training's multi_logloss: 0.110143	training's multi_error: 0.04

2021/12/31 15:30:12 2 [INFO] {'bagging_fraction': 0.8197814508193194, 'feature_fraction': 0.7634908800588147, 'lambda_l1': 0.13532178103709736, 'lambda_l2': 0.21595416946835802, 'learning_rate': 0.01, 'max_leaves': 69, 'metric': ('multi_logloss', 'multi_error'), 'min_data_in_leaf': 31, 'num_classes': 7, 'objective': 'multiclass', 'seed': 0, 'verbose': -1}
/opt/homebrew/lib/python3.9/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(

/opt/homebrew/lib/python3.9/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "

/opt/homebrew/lib/python3.9/site-packages/lightgbm/engine.py:239: UserWarning: 'ver

Training until validation scores don't improve for 20 rounds                    
[50]	training's multi_logloss: 0.438482	training's multi_error: 0.0708359	valid_1's multi_logloss: 0.43934	valid_1's multi_error: 0.0713088
[100]	training's multi_logloss: 0.276366	training's multi_error: 0.0508994	valid_1's multi_logloss: 0.277597	valid_1's multi_error: 0.0512475
[150]	training's multi_logloss: 0.193837	training's multi_error: 0.046145	valid_1's multi_logloss: 0.195396	valid_1's multi_error: 0.0468638
[200]	training's multi_logloss: 0.150607	training's multi_error: 0.0436288	valid_1's multi_logloss: 0.152423	valid_1's multi_error: 0.0443363
[250]	training's multi_logloss: 0.126558	training's multi_error: 0.0418891	valid_1's multi_logloss: 0.128602	valid_1's multi_error: 0.0427825
[300]	training's multi_logloss: 0.111869	training's multi_error: 0.0407081	valid_1's multi_logloss: 0.114136	valid_1's multi_error: 0.0416238
[350]	training's multi_logloss: 0.102321	training's multi_error: 0.039

2021/12/31 15:42:37 2 [INFO] {'bagging_fraction': 0.8752568813438453, 'feature_fraction': 0.7759201601499417, 'lambda_l1': 0.06838213507022683, 'lambda_l2': 0.19405602740685965, 'learning_rate': 0.01, 'max_leaves': 59, 'metric': ('multi_logloss', 'multi_error'), 'min_data_in_leaf': 47, 'num_classes': 7, 'objective': 'multiclass', 'seed': 0, 'verbose': -1}
/opt/homebrew/lib/python3.9/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(

/opt/homebrew/lib/python3.9/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "

/opt/homebrew/lib/python3.9/site-packages/lightgbm/engine.py:239: UserWarning: 'ver

Training until validation scores don't improve for 20 rounds                    
[50]	training's multi_logloss: 0.433631	training's multi_error: 0.0714975	valid_1's multi_logloss: 0.434376	valid_1's multi_error: 0.07185
[100]	training's multi_logloss: 0.272359	training's multi_error: 0.0519913	valid_1's multi_logloss: 0.273444	valid_1's multi_error: 0.0523363
[150]	training's multi_logloss: 0.192102	training's multi_error: 0.0471216	valid_1's multi_logloss: 0.193483	valid_1's multi_error: 0.0477038
[200]	training's multi_logloss: 0.15066	training's multi_error: 0.0444141	valid_1's multi_logloss: 0.152279	valid_1's multi_error: 0.0451587
[250]	training's multi_logloss: 0.127133	training's multi_error: 0.0425897	valid_1's multi_logloss: 0.128971	valid_1's multi_error: 0.0434037
[300]	training's multi_logloss: 0.112867	training's multi_error: 0.0413022	valid_1's multi_logloss: 0.114904	valid_1's multi_error: 0.0422125
[350]	training's multi_logloss: 0.103675	training's multi_error: 0.0402

# Inference

In [23]:
runner = Runner(PROJECT_NAME, ModelLGB, cv=True)
runner.train(X, y, best_params, train_params)

/opt/homebrew/lib/python3.9/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/opt/homebrew/lib/python3.9/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/homebrew/lib/python3.9/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


Training until validation scores don't improve for 20 rounds
[50]	training's multi_logloss: 0.408954	training's multi_error: 0.0651141	valid_1's multi_logloss: 0.409723	valid_1's multi_error: 0.0656763
[100]	training's multi_logloss: 0.248258	training's multi_error: 0.0518156	valid_1's multi_logloss: 0.249324	valid_1's multi_error: 0.0522188
[150]	training's multi_logloss: 0.174568	training's multi_error: 0.0473013	valid_1's multi_logloss: 0.175948	valid_1's multi_error: 0.0478575
[200]	training's multi_logloss: 0.138564	training's multi_error: 0.0447847	valid_1's multi_logloss: 0.140159	valid_1's multi_error: 0.04544
[250]	training's multi_logloss: 0.119885	training's multi_error: 0.0428266	valid_1's multi_logloss: 0.121722	valid_1's multi_error: 0.0436988
[300]	training's multi_logloss: 0.108608	training's multi_error: 0.0413909	valid_1's multi_logloss: 0.110631	valid_1's multi_error: 0.0423588
[350]	training's multi_logloss: 0.101153	training's multi_error: 0.0403241	valid_1's multi

In [12]:
runner.model.feature_importance_

Elevation                             1.974941e+08
class_Wilderness                      1.174233e+07
Horizontal_Distance_To_Fire_Points    1.043957e+07
Horizontal_Distance_To_Roadways       7.581273e+06
class_Soil_Type                       7.397492e+06
diff_roadways_and_hydrology           7.281420e+06
sum_Soil                              5.616168e+06
Vertical_Distance_To_Hydrology        5.329811e+06
important_sum_Soil                    3.069839e+06
Wilderness_Area3                      2.461992e+06
Distance_To_Hydrology                 1.647838e+06
Wilderness_Area1                      1.405506e+06
Soil_Type2                            1.251113e+06
Soil_Type10                           1.006422e+06
Wilderness_Area4                      6.253188e+05
Soil_Type22                           6.194225e+05
Soil_Type4                            6.167165e+05
Soil_Type39                           4.646843e+05
Horizontal_Distance_To_Hydrology      4.441662e+05
Soil_Type3                     

In [19]:
with timer("prediction", logger):
    prob = runner.predict(X_test)
prob

2021/12/31 14:15:27 45 [INFO] [prediction] start.
2021/12/31 14:16:27 47 [INFO] [prediction] done in 59.878 seconds.


array([[6.28273409e-05, 9.99918412e-01, 1.61031304e-05, ...,
        3.48470985e-09, 1.98116223e-06, 6.56874449e-07],
       [1.28247846e-02, 9.87087240e-01, 7.03768914e-05, ...,
        2.55551785e-08, 1.32247441e-05, 4.25129357e-06],
       [4.14478217e-03, 9.95839770e-01, 1.40894491e-05, ...,
        6.37904996e-09, 3.11785180e-07, 1.01346938e-06],
       ...,
       [3.26172074e-04, 9.99628734e-01, 3.53489134e-05, ...,
        8.51956417e-09, 7.72398213e-06, 1.97624459e-06],
       [9.98960357e-01, 7.70648648e-04, 8.86502069e-06, ...,
        9.95778043e-09, 6.77429802e-07, 2.59405881e-04],
       [3.34743790e-04, 1.90647530e-01, 8.08809525e-01, ...,
        2.65922585e-08, 2.03070417e-04, 5.00103587e-06]])

In [20]:
res = pd.DataFrame()
res["Id"] = df_test["Id"]
res["Cover_Type"] = prob.argmax(axis=1) + 1
res.to_csv(f"../submission/submission_{PROJECT_NAME}.csv", index=False)
res

,Id,Cover_Type
0,4000000,2
1,4000001,2
2,4000002,2
3,4000003,2
4,4000004,2
...,...,...
999995,4999995,2
999996,4999996,1
999997,4999997,2
999998,4999998,1


In [21]:
!kaggle competitions submit tabular-playground-series-dec-2021 -f ../submission/submission_{PROJECT_NAME}.csv -m "valid_1's multi_error: 0.0439975"

100%|███████████████████████████████████████| 9.54M/9.54M [00:14<00:00, 688kB/s]
Successfully submitted to Tabular Playground Series - Dec 2021